In [ ]:
# ==========================================
# INTEGRATED CLINICAL TRIAL APP WITH TABLE GENERATOR
# Run this complete notebook in Google Colab
# ==========================================

# CELL 1: Setup and Mount Google Drive
# ==========================================

from google.colab import drive
drive.mount('/content/drive')

import os
project_dir = '/content/drive/MyDrive/ClinicalTrialApp'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

print(f"✅ Google Drive mounted successfully!")
print(f"📁 Project directory: {project_dir}")

!pip install -q pandas numpy plotly sqlalchemy openpyxl gradio google-generativeai faker

print("✅ All packages installed successfully!")

# ==========================================
# CELL 2: Create Database with Sample Data
# ==========================================

import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
import numpy as np

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

def create_clinical_database():
    """Create clinical trial database with sample data"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Drop existing tables if they exist
    cursor.execute('DROP TABLE IF EXISTS patients')
    cursor.execute('DROP TABLE IF EXISTS adverse_events')
    cursor.execute('DROP TABLE IF EXISTS lab_results')
    cursor.execute('DROP TABLE IF EXISTS visits')
    cursor.execute('DROP TABLE IF EXISTS query_log')

    # Create Patients table
    cursor.execute('''
    CREATE TABLE patients (
        patient_id TEXT PRIMARY KEY,
        site_id TEXT,
        age INTEGER,
        gender TEXT,
        enrollment_date DATE,
        treatment_arm TEXT,
        status TEXT
    )
    ''')

    # Create Adverse Events table
    cursor.execute('''
    CREATE TABLE adverse_events (
        event_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        event_term TEXT,
        severity TEXT,
        event_date DATE,
        resolved TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Lab Results table
    cursor.execute('''
    CREATE TABLE lab_results (
        lab_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        test_name TEXT,
        test_value REAL,
        unit TEXT,
        test_date DATE,
        normal_range_low REAL,
        normal_range_high REAL,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Visits table
    cursor.execute('''
    CREATE TABLE visits (
        visit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        visit_number INTEGER,
        visit_date DATE,
        scheduled_date DATE,
        visit_type TEXT,
        completed TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Query Log table
    cursor.execute('''
    CREATE TABLE query_log (
        log_id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp DATETIME,
        natural_language_query TEXT,
        generated_sql TEXT,
        execution_status TEXT,
        results_count INTEGER
    )
    ''')

    # Insert sample patients (50 patients)
    sites = ['SITE001', 'SITE002', 'SITE003', 'SITE004', 'SITE005']
    genders = ['M', 'F']
    treatment_arms = ['Treatment A', 'Treatment B', 'Placebo']
    statuses = ['Active', 'Completed', 'Withdrawn']

    patients_data = []
    for i in range(1, 51):
        patient_id = f'PT{i:04d}'
        site_id = random.choice(sites)
        age = random.randint(25, 75)
        gender = random.choice(genders)
        enrollment_date = (datetime.now() - timedelta(days=random.randint(30, 365))).strftime('%Y-%m-%d')
        treatment_arm = random.choice(treatment_arms)
        status = random.choice(statuses)
        patients_data.append((patient_id, site_id, age, gender, enrollment_date, treatment_arm, status))

    cursor.executemany('INSERT INTO patients VALUES (?, ?, ?, ?, ?, ?, ?)', patients_data)

    # Insert adverse events
    adverse_events_data = []
    event_terms = ['Headache', 'Nausea', 'Fatigue', 'Dizziness', 'Rash', 'Fever', 'Insomnia', 'Chest Pain']
    severities = ['Mild', 'Moderate', 'Severe', None]

    for patient in patients_data[:35]:
        num_events = random.randint(1, 4)
        for j in range(num_events):
            patient_id = patient[0]
            event_term = random.choice(event_terms)
            severity = random.choice(severities)
            event_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            resolved = random.choice(['Yes', 'No', None])
            adverse_events_data.append((patient_id, event_term, severity, event_date, resolved))

    cursor.executemany('INSERT INTO adverse_events (patient_id, event_term, severity, event_date, resolved) VALUES (?, ?, ?, ?, ?)',
                       adverse_events_data)

    # Insert lab results
    lab_tests = [
        ('Hemoglobin', 'g/dL', 12.0, 16.0),
        ('White Blood Cell Count', '10^9/L', 4.0, 11.0),
        ('Platelet Count', '10^9/L', 150, 400),
        ('ALT', 'U/L', 7, 56),
        ('AST', 'U/L', 10, 40),
        ('Creatinine', 'mg/dL', 0.7, 1.3),
        ('Glucose', 'mg/dL', 70, 100),
        ('Sodium', 'mmol/L', 135, 145),
        ('Potassium', 'mmol/L', 3.5, 5.0)
    ]

    lab_results_data = []
    for patient in patients_data:
        num_labs = random.randint(5, 10)
        for _ in range(num_labs):
            patient_id = patient[0]
            test = random.choice(lab_tests)
            test_name = test[0]
            unit = test[1]
            normal_low = test[2]
            normal_high = test[3]

            rand_val = random.random()
            if rand_val < 0.75:
                test_value = round(random.uniform(normal_low, normal_high), 2)
            elif rand_val < 0.95:
                test_value = round(random.uniform(normal_high, normal_high * 1.5), 2)
            else:
                test_value = round(random.uniform(normal_low * 0.5, normal_low), 2)

            test_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            lab_results_data.append((patient_id, test_name, test_value, unit, test_date, normal_low, normal_high))

    cursor.executemany('INSERT INTO lab_results (patient_id, test_name, test_value, unit, test_date, normal_range_low, normal_range_high) VALUES (?, ?, ?, ?, ?, ?, ?)',
                       lab_results_data)

    # Insert visits
    visits_data = []
    for patient in patients_data:
        for visit_num in range(1, 5):
            patient_id = patient[0]
            scheduled_date = datetime.strptime(patient[4], '%Y-%m-%d') + timedelta(days=30*visit_num)

            if random.random() < 0.65:
                visit_date = scheduled_date + timedelta(days=random.randint(-2, 2))
            else:
                visit_date = scheduled_date + timedelta(days=random.randint(8, 30))

            visit_type = f'Visit {visit_num}'
            completed = random.choice(['Yes', 'No', None])
            visits_data.append((patient_id, visit_num, visit_date.strftime('%Y-%m-%d'),
                              scheduled_date.strftime('%Y-%m-%d'), visit_type, completed))

    cursor.executemany('INSERT INTO visits (patient_id, visit_number, visit_date, scheduled_date, visit_type, completed) VALUES (?, ?, ?, ?, ?, ?)',
                       visits_data)

    conn.commit()
    conn.close()

    print("✅ Clinical trial database created successfully!")
    print(f"📁 Database location: {DB_PATH}")
    print(f"   - {len(patients_data)} patients")
    print(f"   - {len(adverse_events_data)} adverse events")
    print(f"   - {len(lab_results_data)} lab results")
    print(f"   - {len(visits_data)} visits")

    return True

create_clinical_database()

conn = sqlite3.connect(DB_PATH)
print("\n📊 Sample Patients:")
display(pd.read_sql_query("SELECT * FROM patients LIMIT 5", conn))
conn.close()

print("\n💾 Database is now permanently stored in your Google Drive!")

# ==========================================
# CELL 3: Create Dynamic Table Generator Module
# ==========================================

table_generator_code = '''from faker import Faker
import pandas as pd
import sqlite3
import random
from typing import Dict, List, Tuple

class DynamicTableGenerator:
    """Generate custom tables dynamically based on user specifications"""

    def __init__(self, db_path):
        self.db_path = db_path
        self.fake = Faker()
        Faker.seed(0)
        random.seed(0)

    def create_table_from_schema(self, table_name: str, schema: Dict[str, str]) -> Tuple[bool, str]:
        """Create a new table in the database
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table already exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' already exists!"

            # Build column definitions
            cols_def = []
            for col, dtype in schema.items():
                dtype_sql = "INTEGER" if dtype.upper() == "BOOLEAN" else dtype.upper()
                cols_def.append(f'"{col}" {dtype_sql}')

            create_sql = f'CREATE TABLE "{table_name}" ({", ".join(cols_def)});'
            cursor.execute(create_sql)
            conn.commit()
            conn.close()

            return True, f"✅ Table '{table_name}' created successfully!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def generate_value(self, dtype: str, col_name: str = None):
        """Generate fake data based on data type and column name"""
        dtype = dtype.upper()

        if "INT" in dtype:
            return random.randint(1, 1000)
        if "REAL" in dtype or "FLOAT" in dtype or "DOUBLE" in dtype:
            return round(random.uniform(1, 1000), 3)
        if "DATE" in dtype:
            return self.fake.date()
        if "BOOL" in dtype or "BOOLEAN" in dtype:
            return random.choice([0, 1])

        # Heuristics for TEXT columns
        if col_name:
            lname = col_name.lower()
            if "name" in lname:
                return self.fake.name()
            if "email" in lname:
                return self.fake.email()
            if "phone" in lname or "mobile" in lname:
                return self.fake.phone_number()
            if "address" in lname:
                return self.fake.address().replace("\\n", " ")
            if "city" in lname:
                return self.fake.city()
            if "country" in lname:
                return self.fake.country()
            if "desc" in lname or "text" in lname or "note" in lname:
                return self.fake.sentence(nb_words=8)

        return self.fake.word()

    def insert_data(self, table_name: str, schema: Dict[str, str], num_rows: int) -> Tuple[bool, str]:
        """Insert fake data into a table
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
            num_rows: Number of rows to generate
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if not cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' does not exist!"

            # Generate data
            rows = []
            col_names = list(schema.keys())
            for i in range(num_rows):
                row = [self.generate_value(schema[c], c) for c in col_names]
                rows.append(tuple(row))

            # Insert data
            placeholders = ", ".join(["?"] * len(col_names))
            insert_sql = f'INSERT INTO "{table_name}" ({", ".join(["\\"" + c + "\\"" for c in col_names])}) VALUES ({placeholders});'
            cursor.executemany(insert_sql, rows)
            conn.commit()
            conn.close()

            return True, f"✅ Inserted {num_rows} rows into '{table_name}'!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def get_all_tables(self) -> List[str]:
        """Get list of all tables in database"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
            tables = [row[0] for row in cursor.fetchall()]
            conn.close()
            return tables
        except Exception as e:
            return []

    def get_table_info(self, table_name: str) -> pd.DataFrame:
        """Get information about a table"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(f"PRAGMA table_info('{table_name}')", conn)
            conn.close()
            return df
        except Exception as e:
            return pd.DataFrame()

print("✅ Dynamic Table Generator module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/table_generator.py', 'w') as f:
    f.write(table_generator_code)

print("✅ table_generator.py saved successfully!")

# ==========================================
# CELL 4: Create All Other Modules (NL-to-SQL, Data Quality, Query Generator)
# ==========================================

# (Keep existing modules - nl_to_sql.py, data_quality_checker.py, query_generator.py)
# Copy from your existing code...

nl_to_sql_code = '''import re
from typing import List, Tuple, Dict
import sqlite3
import pandas as pd
from datetime import datetime
import google.generativeai as genai

class SimpleNLtoSQL:
    """Gemini-powered NL to SQL converter for clinical trial queries"""

    def __init__(self, db_path, api_key=None):
        self.db_path = db_path
        self.api_key = api_key

        if self.api_key:
            try:
                genai.configure(api_key=self.api_key)
                self.model = genai.GenerativeModel("models/gemini-2.0-flash-exp")
                print("✅ Gemini API configured successfully!")
            except Exception as e:
                print(f"⚠️ Error configuring Gemini: {e}")
                self.model = None
        else:
            self.model = None

        self.schema_info = """
DATABASE SCHEMA FOR CLINICAL TRIAL:

1. patients table:
   - patient_id (TEXT PRIMARY KEY)
   - site_id (TEXT)
   - age (INTEGER)
   - gender (TEXT)
   - enrollment_date (DATE)
   - treatment_arm (TEXT)
   - status (TEXT)

2. adverse_events table:
   - event_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - event_term (TEXT)
   - severity (TEXT)
   - event_date (DATE)
   - resolved (TEXT)

3. lab_results table:
   - lab_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - test_name (TEXT)
   - test_value (REAL)
   - unit (TEXT)
   - test_date (DATE)
   - normal_range_low (REAL)
   - normal_range_high (REAL)

4. visits table:
   - visit_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - visit_number (INTEGER)
   - visit_date (DATE)
   - scheduled_date (DATE)
   - visit_type (TEXT)
   - completed (TEXT)
"""

    def parse_query(self, nl_query: str) -> Tuple[str, str, str]:
        """Convert natural language to SQL using Gemini API"""
        nl_query = nl_query.strip()

        if not nl_query:
            return None, "error", "Please enter a query."

        if not self.model:
            return None, "error", "⚠️ Gemini API not configured. Please provide API key in the app."

        try:
            prompt = f"""You are a SQL expert for a clinical trial database.

{self.schema_info}

IMPORTANT RULES:
1. Generate ONLY valid SQLite syntax
2. Return ONLY the SQL query, no explanations or markdown
3. Use proper table and column names from the schema above
4. Always add LIMIT 100 to prevent large result sets
5. For "outliers" or "out of range" labs, use: test_value < normal_range_low OR test_value > normal_range_high

USER QUERY: "{nl_query}"

Generate the SQL query:"""

            response = self.model.generate_content(prompt)

            if not response or not response.text:
                return None, "error", "No response from Gemini API"

            sql_query = self._extract_sql(response.text)

            if not sql_query:
                return None, "error", "Could not extract valid SQL from response"

            is_valid, validation_msg = self._validate_sql(sql_query)

            if not is_valid:
                return None, "error", f"Invalid SQL: {validation_msg}"

            explanation = f"Natural Language: {nl_query}\\n\\nGenerated SQL Query:\\n{sql_query}"

            return sql_query, "success", explanation

        except Exception as e:
            return None, "error", f"Error generating SQL: {str(e)}"

    def _extract_sql(self, response_text: str) -> str:
        """Extract SQL query from Gemini response"""
        response_text = response_text.strip()

        if response_text.startswith('```'):
            lines = response_text.split('\\n')
            lines = lines[1:]
            if lines and lines[-1].strip() == '```':
                lines = lines[:-1]
            response_text = '\\n'.join(lines)

        sql_query = response_text.strip()

        if not sql_query.upper().startswith('SELECT'):
            match = re.search(r'(SELECT\\s+.+)', sql_query, re.IGNORECASE | re.DOTALL)
            if match:
                sql_query = match.group(1)

        return sql_query

    def _validate_sql(self, sql_query: str) -> Tuple[bool, str]:
        """Validate SQL query"""
        if not sql_query.upper().strip().startswith('SELECT'):
            return False, "Only SELECT queries are allowed"

        dangerous_keywords = ['DROP', 'DELETE', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE']
        upper_query = sql_query.upper()
        for keyword in dangerous_keywords:
            if keyword in upper_query:
                return False, f"Query contains forbidden keyword: {keyword}"

        return True, "Valid"

    def execute_query(self, sql_query: str) -> Tuple:
        """Execute SQL query and return results"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(sql_query, conn)
            conn.close()
            return df, "success"
        except Exception as e:
            return None, f"Error executing query: {str(e)}"

    def get_example_queries(self) -> List[str]:
        """Return list of example queries"""
        return [
            "Show all patients",
            "How many patients per site?",
            "List patients with severe adverse events",
            "Show patients older than 65",
            "Find lab results with outliers",
            "Count adverse events by severity"
        ]

print("✅ Gemini-powered NL-to-SQL module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/nl_to_sql.py', 'w') as f:
    f.write(nl_to_sql_code)

# (Add data_quality_checker.py and query_generator.py - keeping your existing code)

print("✅ All modules created successfully!")

# ==========================================
# CELL 5: Create Enhanced Gradio App with Table Generator
# ==========================================

app_code = '''import gradio as gr
import sqlite3
import pandas as pd
import plotly.express as px
from datetime import datetime
import sys
import os

sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from nl_to_sql import SimpleNLtoSQL
from table_generator import DynamicTableGenerator

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

nl_sql = None
table_gen = DynamicTableGenerator(DB_PATH)

def configure_api(api_key):
    global nl_sql
    if not api_key or api_key.strip() == "":
        return "⚠️ Please enter your Gemini API key", False

    try:
        nl_sql = SimpleNLtoSQL(DB_PATH, api_key=api_key.strip())
        return "✅ Gemini API configured! You can now use the query interface.", True
    except Exception as e:
        return f"❌ Error: {str(e)}", False

def execute_nl_query(user_query, confirmation):
    global nl_sql

    if not nl_sql:
        return "⚠️ Please configure Gemini API first!", None, "", gr.update(visible=False)

    if not user_query or user_query.strip() == "":
        return "Please enter a query.", None, "", gr.update(visible=False)

    try:
        sql_query, status, explanation = nl_sql.parse_query(user_query)

        if status == "success" and sql_query:
            if not confirmation:
                return (
                    f"🔍 Generated SQL Query:\\n\\n{sql_query}\\n\\n" +
                    f"💡 Explanation:\\n{explanation}\\n\\n" +
                    "⚠️ Please review and check the confirmation box to execute.",
                    None,
                    sql_query,
                    gr.update(visible=True)
                )

            df_result, exec_status = nl_sql.execute_query(sql_query)

            if exec_status == "success":
                result_text = f"✅ Query executed!\\n\\n📊 Found {len(df_result)} results"
                return result_text, df_result, sql_query, gr.update(visible=False)
            else:
                return f"❌ Error: {exec_status}", None, sql_query, gr.update(visible=False)
        else:
            return f"⚠️ {explanation}", None, sql_query or "", gr.update(visible=False)

    except Exception as e:
        return f"❌ Error: {str(e)}", None, "", gr.update(visible=False)

def create_custom_table(table_name, columns_text, num_rows):
    """Create a new custom table"""
    if not table_name or not columns_text:
        return "⚠️ Please provide table name and columns!", None

    # Parse columns (format: col1:TEXT, col2:INTEGER, col3:DATE)
    try:
        schema = {}
        for line in columns_text.strip().split('\\n'):
            if ':' in line:
                col, dtype = line.split(':')
                schema[col.strip()] = dtype.strip()

        if not schema:
            return "⚠️ Invalid column format! Use: column_name:DATA_TYPE (one per line)", None

        # Create table
        success, msg = table_gen.create_table_from_schema(table_name, schema)
        if not success:
            return msg, None

        # Insert data
        success, data_msg = table_gen.insert_data(table_name, schema, int(num_rows))

        # Show preview
        conn = sqlite3.connect(DB_PATH)
        df = pd.read_sql_query(f'SELECT * FROM "{table_name}" LIMIT 10', conn)
        conn.close()

        return f"{msg}\\n{data_msg}\\n\\n✅ Table created with {num_rows} rows!", df

    except Exception as e:
        return f"❌ Error: {str(e)}", None

def list_all_tables():
    """List all tables in database"""
    tables = table_gen.get_all_tables()
    return "\\n".join([f"• {t}" for t in tables])

with gr.Blocks(title="Clinical Trial Data Management", theme=gr.themes.Soft()) as app:

    gr.Markdown("""
    # 🏥 Clinical Trial Data Management System
    ### AI-Powered with Google Gemini + Dynamic Table Generator
    **Version 3.0**
    """)

    with gr.Tab("🔑 API Configuration"):
        gr.Markdown("### Configure Gemini API")

        api_key_input = gr.Textbox(
            label="🔑 Enter Gemini API Key",
            type="password",
            placeholder="Get from https://aistudio.google.com/apikey"
        )
        api_config_btn = gr.Button("⚙️ Configure API", variant="primary")
        api_status = gr.Textbox(label="API Status", lines=2)
        api_configured = gr.State(False)

        api_config_btn.click(
            fn=configure_api,
            inputs=[api_key_input],
            outputs=[api_status, api_configured]
        )

    with gr.Tab("🔍 Query Interface"):
        gr.Markdown("### Ask questions in plain English - Powered by Gemini 2.0")

        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Your question:",
                    placeholder="e.g., Show patients with severe adverse events",
                    lines=2
                )

                confirmation_checkbox = gr.Checkbox(
                    label="✅ I reviewed the SQL and want to execute it",
                    value=False,
                    visible=False
                )

                with gr.Row():
                    query_btn = gr.Button("🚀 Generate SQL", variant="primary")
                    clear_btn = gr.Button("🗑️ Clear")

        query_output = gr.Textbox(label="Results", lines=8)
        sql_output = gr.Code(label="Generated SQL", language="sql")
        results_df = gr.Dataframe(label="Data", wrap=True)

        query_btn.click(
            fn=execute_nl_query,
            inputs=[query_input, confirmation_checkbox],
            outputs=[query_output, results_df, sql_output, confirmation_checkbox]
        )

        clear_btn.click(
            fn=lambda: ("", None, "", False, gr.update(visible=False)),
            outputs=[query_output, results_df, sql_output, confirmation_checkbox, confirmation_checkbox]
        )

    with gr.Tab("🆕 Create Custom Tables"):
        gr.Markdown("""
        ### 🎨 Dynamic Table Generator
        Create your own custom tables with automatic fake data generation!
        """)

        with gr.Row():
            with gr.Column():
                new_table_name = gr.Textbox(
                    label="📋 Table Name",
                    placeholder="e.g., custom_data"
                )

                columns_input = gr.Textbox(
                    label="📝 Columns (one per line)",
                    placeholder="column1:TEXT\\ncolumn2:INTEGER\\ncolumn3:DATE\\ncolumn4:REAL",
                    lines=8
                )

                gr.Markdown("""
                **Supported Data Types:**
                - TEXT (for names, emails, addresses, etc.)
                - INTEGER (for numbers)
                - REAL (for decimals)
                - DATE (for dates)
                - BOOLEAN (for yes/no)

                **Smart Detection:** Column names like "name", "email", "phone", "address" will auto-generate appropriate data!
                """)

                num_rows_input = gr.Slider(
                    minimum=1,
                    maximum=1000,
                    value=100,
                    step=1,
                    label="📊 Number of Rows to Generate"
                )

                create_table_btn = gr.Button("🎯 Create Table & Generate Data", variant="primary")

            with gr.Column():
                gr.Markdown("### 📚 Existing Tables")
                tables_list = gr.Textbox(
                    label="Current Tables in Database",
                    lines=15,
                    interactive=False
                )
                refresh_tables_btn = gr.Button("🔄 Refresh List")

        create_output = gr.Textbox(label="Status", lines=5)
        preview_df = gr.Dataframe(label="Preview (First 10 Rows)", wrap=True)

        create_table_btn.click(
            fn=create_custom_table,
            inputs=[new_table_name, columns_input, num_rows_input],
            outputs=[create_output, preview_df]
        )

        refresh_tables_btn.click(
            fn=list_all_tables,
            outputs=[tables_list]
        )

        # Auto-load tables on tab load
        app.load(fn=list_all_tables, outputs=[tables_list])

    with gr.Tab("📊 Dashboard"):
        gr.Markdown("### Analytics & Metrics")

        def create_dashboard():
            try:
                conn = sqlite3.connect(DB_PATH)

                # Get stats
                stats = {
                    'total_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients", conn).iloc[0]['count'],
                    'active_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients WHERE status = 'Active'", conn).iloc[0]['count'],
                    'total_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events", conn).iloc[0]['count'],
                    'severe_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events WHERE severity = 'Severe'", conn).iloc[0]['count'],
                }

                summary = f"""
CLINICAL TRIAL DASHBOARD
Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

KEY METRICS:

👥 Patients:
   • Total: {stats['total_patients']}
   • Active: {stats['active_patients']} ({stats['active_patients']/stats['total_patients']*100:.1f}%)

⚠️ Safety:
   • Total AEs: {stats['total_aes']}
   • Severe AEs: {stats['severe_aes']}
"""

                site_dist = pd.read_sql_query("SELECT site_id, COUNT(*) as count FROM patients GROUP BY site_id", conn)
                fig1 = px.bar(site_dist, x='site_id', y='count', title="Patients by Site")

                treatment_dist = pd.read_sql_query("SELECT treatment_arm, COUNT(*) as count FROM patients GROUP BY treatment_arm", conn)
                fig2 = px.pie(treatment_dist, values='count', names='treatment_arm', title="Treatment Arms")

                conn.close()

                return summary, fig1, fig2

            except Exception as e:
                return f"❌ Error: {str(e)}", None, None

        refresh_btn = gr.Button("🔄 Refresh", variant="primary")

        dashboard_summary = gr.Textbox(label="Metrics", lines=20)

        with gr.Row():
            chart1 = gr.Plot(label="Patients by Site")
            chart2 = gr.Plot(label="Treatment Arms")

        refresh_btn.click(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

        app.load(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

    gr.Markdown("""
    ---
    **v3.0** | Powered by Google Gemini | Data in Google Drive | Dynamic Table Generator

    💾 Location: `/content/drive/MyDrive/ClinicalTrialApp/`

    🎯 **New Feature**: Create custom tables with automatic fake data generation!
    """)

print("✅ Enhanced Gradio app created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/app_gradio.py', 'w') as f:
    f.write(app_code)

print("✅ app_gradio.py saved successfully!")

# ==========================================
# CELL 6: Instructions
# ==========================================

print("""
╔═══════════════════════════════════════════════════════════╗
║    🎉 INTEGRATED APP WITH TABLE GENERATOR READY!          ║
╚═══════════════════════════════════════════════════════════╝

📝 HOW TO GET YOUR GEMINI API KEY:

1. 🌐 Go to: https://aistudio.google.com/apikey
2. 🔑 Click "Create API Key" button
3. 📋 Copy your API key
4. ✅ Paste it in the app when it launches
5. ⚙️ Click "Configure API" button

🎯 NEW FEATURES:
   ✅ AI SQL generation (Gemini 2.0)
   ✅ Query confirmation (safety first!)
   ✅ Interactive dashboard
   ✨ Dynamic table creation with fake data!
   ✨ Custom schema definition
   ✨ Automatic data type detection
   ✨ Real-time table listing

💾 DATA: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this notebook running!
""")

# ==========================================
# CELL 7: Launch the App
# ==========================================

import os
os.chdir('/content/drive/MyDrive/ClinicalTrialApp')

import sys
sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from app_gradio import app

app.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("""
╔═══════════════════════════════════════════════════════════╗
║  🎉 APP IS RUNNING!                                       ║
╚═══════════════════════════════════════════════════════════╝

🔑 SETUP:
   1. Click the public URL above
   2. Enter your Gemini API key in "API Configuration" tab
   3. Click "Configure API"
   4. Start using all features!

✨ TABS AVAILABLE:
   1️⃣ API Configuration - Setup Gemini
   2️⃣ Query Interface - Ask questions in plain English
   3️⃣ Create Custom Tables - NEW! Generate tables dynamically
   4️⃣ Dashboard - View analytics

🎨 EXAMPLE: CREATE CUSTOM TABLE
   Table Name: employee_data
   Columns:
   emp_name:TEXT
   emp_email:TEXT
   emp_age:INTEGER
   emp_salary:REAL
   hire_date:DATE

   Then set rows to 100 and click "Create Table & Generate Data"!

💾 DATA SAVED IN: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this cell running to use the app!

🔄 To restart: Run Cell 7 again
""")

Mounted at /content/drive
✅ Google Drive mounted successfully!
📁 Project directory: /content/drive/MyDrive/ClinicalTrialApp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 19.5 MB/s eta 0:00:00
✅ All packages installed successfully!
✅ Clinical trial database created successfully!
📁 Database location: /content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db
   - 50 patients
   - 80 adverse events
   - 387 lab results
   - 200 visits

📊 Sample Patients:


,patient_id,site_id,age,gender,enrollment_date,treatment_arm,status
0,PT0001,SITE004,49,F,2025-03-17,Placebo,Withdrawn
1,PT0002,SITE003,48,M,2025-05-10,Treatment A,Withdrawn
2,PT0003,SITE005,35,F,2025-09-17,Placebo,Withdrawn
3,PT0004,SITE001,25,F,2025-02-03,Treatment A,Completed
4,PT0005,SITE001,46,M,2025-03-23,Placebo,Completed



💾 Database is now permanently stored in your Google Drive!
✅ table_generator.py saved successfully!
✅ All modules created successfully!
✅ app_gradio.py saved successfully!

╔═══════════════════════════════════════════════════════════╗
║    🎉 INTEGRATED APP WITH TABLE GENERATOR READY!          ║
╚═══════════════════════════════════════════════════════════╝

📝 HOW TO GET YOUR GEMINI API KEY:

1. 🌐 Go to: https://aistudio.google.com/apikey
2. 🔑 Click "Create API Key" button
3. 📋 Copy your API key
4. ✅ Paste it in the app when it launches
5. ⚙️ Click "Configure API" button

🎯 NEW FEATURES:
   ✅ AI SQL generation (Gemini 2.0)
   ✅ Query confirmation (safety first!)
   ✅ Interactive dashboard
   ✨ Dynamic table creation with fake data!
   ✨ Custom schema definition
   ✨ Automatic data type detection
   ✨ Real-time table listing

💾 DATA: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this notebook running!

✅ Gemini-powered NL-to-SQL module created!
✅ Dynamic Table Generator module create

✅ Gemini API configured successfully!


Keyboard interruption in main thread... closing server.
Killing tunnel 0.0.0.0:7860 <> https://2cebafb51596da9f0c.gradio.live

╔═══════════════════════════════════════════════════════════╗
║  🎉 APP IS RUNNING!                                       ║
╚═══════════════════════════════════════════════════════════╝

🔑 SETUP:
   1. Click the public URL above
   2. Enter your Gemini API key in "API Configuration" tab
   3. Click "Configure API"
   4. Start using all features!

✨ TABS AVAILABLE:
   1️⃣ API Configuration - Setup Gemini
   2️⃣ Query Interface - Ask questions in plain English
   3️⃣ Create Custom Tables - NEW! Generate tables dynamically
   4️⃣ Dashboard - View analytics

🎨 EXAMPLE: CREATE CUSTOM TABLE
   Table Name: employee_data
   Columns:
   emp_name:TEXT
   emp_email:TEXT
   emp_age:INTEGER
   emp_salary:REAL
   hire_date:DATE

   Then set rows to 100 and click "Create Table & Generate Data"!

💾 DATA SAVED IN: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this cell runni

In [ ]:

# ==========================================
# CELL 7: Launch the App
# ==========================================

import os
os.chdir('/content/drive/MyDrive/ClinicalTrialApp')

import sys
sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from app_gradio import app

app.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("""
╔═══════════════════════════════════════════════════════════╗
║  🎉 APP IS RUNNING!                                       ║
╚═══════════════════════════════════════════════════════════╝

🔑 SETUP:
   1. Click the public URL above
   2. Enter your Gemini API key in "API Configuration" tab
   3. Click "Configure API"
   4. Start using all features!

✨ TABS AVAILABLE:
   1️⃣ API Configuration - Setup Gemini
   2️⃣ Query Interface - Ask questions in plain English
   3️⃣ Create Custom Tables - NEW! Generate tables dynamically
   4️⃣ Dashboard - View analytics

🎨 EXAMPLE: CREATE CUSTOM TABLE
   Table Name: employee_data
   Columns:
   emp_name:TEXT
   emp_email:TEXT
   emp_age:INTEGER
   emp_salary:REAL
   hire_date:DATE

   Then set rows to 100 and click "Create Table & Generate Data"!

💾 DATA SAVED IN: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this cell running to use the app!

🔄 To restart: Run Cell 7 again
""")

Rerunning server... use `close()` to stop if you need to change `launch()` parameters.
----
Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://2cebafb51596da9f0c.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# ==========================================
# INTEGRATED CLINICAL TRIAL APP WITH TABLE GENERATOR
# Run this complete notebook in Google Colab
# ==========================================

# CELL 1: Setup and Mount Google Drive
# ==========================================

from google.colab import drive
drive.mount('/content/drive')

import os
project_dir = '/content/drive/MyDrive/ClinicalTrialApp'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

print(f"✅ Google Drive mounted successfully!")
print(f"📁 Project directory: {project_dir}")

!pip install -q pandas numpy plotly sqlalchemy openpyxl gradio google-generativeai faker

print("✅ All packages installed successfully!")

# ==========================================
# CELL 2: Create Database with Sample Data
# ==========================================

import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
import numpy as np

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

def create_clinical_database():
    """Create clinical trial database with sample data"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Drop existing tables if they exist
    cursor.execute('DROP TABLE IF EXISTS patients')
    cursor.execute('DROP TABLE IF EXISTS adverse_events')
    cursor.execute('DROP TABLE IF EXISTS lab_results')
    cursor.execute('DROP TABLE IF EXISTS visits')
    cursor.execute('DROP TABLE IF EXISTS query_log')

    # Create Patients table
    cursor.execute('''
    CREATE TABLE patients (
        patient_id TEXT PRIMARY KEY,
        site_id TEXT,
        age INTEGER,
        gender TEXT,
        enrollment_date DATE,
        treatment_arm TEXT,
        status TEXT
    )
    ''')

    # Create Adverse Events table
    cursor.execute('''
    CREATE TABLE adverse_events (
        event_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        event_term TEXT,
        severity TEXT,
        event_date DATE,
        resolved TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Lab Results table
    cursor.execute('''
    CREATE TABLE lab_results (
        lab_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        test_name TEXT,
        test_value REAL,
        unit TEXT,
        test_date DATE,
        normal_range_low REAL,
        normal_range_high REAL,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Visits table
    cursor.execute('''
    CREATE TABLE visits (
        visit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        visit_number INTEGER,
        visit_date DATE,
        scheduled_date DATE,
        visit_type TEXT,
        completed TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Query Log table
    cursor.execute('''
    CREATE TABLE query_log (
        log_id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp DATETIME,
        natural_language_query TEXT,
        generated_sql TEXT,
        execution_status TEXT,
        results_count INTEGER
    )
    ''')

    # Insert sample patients (50 patients)
    sites = ['SITE001', 'SITE002', 'SITE003', 'SITE004', 'SITE005']
    genders = ['M', 'F']
    treatment_arms = ['Treatment A', 'Treatment B', 'Placebo']
    statuses = ['Active', 'Completed', 'Withdrawn']

    patients_data = []
    for i in range(1, 51):
        patient_id = f'PT{i:04d}'
        site_id = random.choice(sites)
        # More realistic age distribution with varied range
        age_range = random.choice([
            (18, 30),   # Young adults
            (31, 45),   # Adults
            (46, 60),   # Middle-aged
            (61, 75),   # Senior
            (76, 85)    # Elderly
        ])
        age = random.randint(age_range[0], age_range[1])
        gender = random.choice(genders)
        enrollment_date = (datetime.now() - timedelta(days=random.randint(30, 365))).strftime('%Y-%m-%d')
        treatment_arm = random.choice(treatment_arms)
        status = random.choice(statuses)
        patients_data.append((patient_id, site_id, age, gender, enrollment_date, treatment_arm, status))

    cursor.executemany('INSERT INTO patients VALUES (?, ?, ?, ?, ?, ?, ?)', patients_data)

    # Insert adverse events
    adverse_events_data = []
    event_terms = ['Headache', 'Nausea', 'Fatigue', 'Dizziness', 'Rash', 'Fever', 'Insomnia', 'Chest Pain']
    severities = ['Mild', 'Moderate', 'Severe', None]

    for patient in patients_data[:35]:
        num_events = random.randint(1, 4)
        for j in range(num_events):
            patient_id = patient[0]
            event_term = random.choice(event_terms)
            severity = random.choice(severities)
            event_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            resolved = random.choice(['Yes', 'No', None])
            adverse_events_data.append((patient_id, event_term, severity, event_date, resolved))

    cursor.executemany('INSERT INTO adverse_events (patient_id, event_term, severity, event_date, resolved) VALUES (?, ?, ?, ?, ?)',
                       adverse_events_data)

    # Insert lab results
    lab_tests = [
        ('Hemoglobin', 'g/dL', 12.0, 16.0),
        ('White Blood Cell Count', '10^9/L', 4.0, 11.0),
        ('Platelet Count', '10^9/L', 150, 400),
        ('ALT', 'U/L', 7, 56),
        ('AST', 'U/L', 10, 40),
        ('Creatinine', 'mg/dL', 0.7, 1.3),
        ('Glucose', 'mg/dL', 70, 100),
        ('Sodium', 'mmol/L', 135, 145),
        ('Potassium', 'mmol/L', 3.5, 5.0)
    ]

    lab_results_data = []
    for patient in patients_data:
        num_labs = random.randint(5, 10)
        for _ in range(num_labs):
            patient_id = patient[0]
            test = random.choice(lab_tests)
            test_name = test[0]
            unit = test[1]
            normal_low = test[2]
            normal_high = test[3]

            rand_val = random.random()
            if rand_val < 0.75:
                test_value = round(random.uniform(normal_low, normal_high), 2)
            elif rand_val < 0.95:
                test_value = round(random.uniform(normal_high, normal_high * 1.5), 2)
            else:
                test_value = round(random.uniform(normal_low * 0.5, normal_low), 2)

            test_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            lab_results_data.append((patient_id, test_name, test_value, unit, test_date, normal_low, normal_high))

    cursor.executemany('INSERT INTO lab_results (patient_id, test_name, test_value, unit, test_date, normal_range_low, normal_range_high) VALUES (?, ?, ?, ?, ?, ?, ?)',
                       lab_results_data)

    # Insert visits
    visits_data = []
    for patient in patients_data:
        for visit_num in range(1, 5):
            patient_id = patient[0]
            scheduled_date = datetime.strptime(patient[4], '%Y-%m-%d') + timedelta(days=30*visit_num)

            if random.random() < 0.65:
                visit_date = scheduled_date + timedelta(days=random.randint(-2, 2))
            else:
                visit_date = scheduled_date + timedelta(days=random.randint(8, 30))

            visit_type = f'Visit {visit_num}'
            completed = random.choice(['Yes', 'No', None])
            visits_data.append((patient_id, visit_num, visit_date.strftime('%Y-%m-%d'),
                              scheduled_date.strftime('%Y-%m-%d'), visit_type, completed))

    cursor.executemany('INSERT INTO visits (patient_id, visit_number, visit_date, scheduled_date, visit_type, completed) VALUES (?, ?, ?, ?, ?, ?)',
                       visits_data)

    conn.commit()
    conn.close()

    print("✅ Clinical trial database created successfully!")
    print(f"📁 Database location: {DB_PATH}")
    print(f"   - {len(patients_data)} patients")
    print(f"   - {len(adverse_events_data)} adverse events")
    print(f"   - {len(lab_results_data)} lab results")
    print(f"   - {len(visits_data)} visits")

    return True

create_clinical_database()

conn = sqlite3.connect(DB_PATH)
print("\n📊 Sample Patients:")
display(pd.read_sql_query("SELECT * FROM patients LIMIT 5", conn))
conn.close()

print("\n💾 Database is now permanently stored in your Google Drive!")

# ==========================================
# CELL 3: Create Dynamic Table Generator Module
# ==========================================

table_generator_code = '''from faker import Faker
import pandas as pd
import sqlite3
import random
from typing import Dict, List, Tuple

class DynamicTableGenerator:
    """Generate custom tables dynamically based on user specifications"""

    def __init__(self, db_path):
        self.db_path = db_path
        self.fake = Faker()
        Faker.seed(0)
        random.seed(0)

    def create_table_from_schema(self, table_name: str, schema: Dict[str, str]) -> Tuple[bool, str]:
        """Create a new table in the database
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table already exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' already exists!"

            # Build column definitions
            cols_def = []
            for col, dtype in schema.items():
                dtype_sql = "INTEGER" if dtype.upper() == "BOOLEAN" else dtype.upper()
                cols_def.append(f'"{col}" {dtype_sql}')

            create_sql = f'CREATE TABLE "{table_name}" ({", ".join(cols_def)});'
            cursor.execute(create_sql)
            conn.commit()
            conn.close()

            return True, f"✅ Table '{table_name}' created successfully!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def generate_value(self, dtype: str, col_name: str = None):
        """Generate fake data based on data type and column name"""
        dtype = dtype.upper()

        if "INT" in dtype:
            # Smart integer generation based on column name
            if col_name:
                lname = col_name.lower()
                if "age" in lname:
                    return random.randint(18, 85)
                elif "year" in lname:
                    return random.randint(1950, 2025)
                elif "quantity" in lname or "count" in lname:
                    return random.randint(1, 100)
                elif "id" in lname:
                    return random.randint(1000, 9999)
                elif "salary" in lname or "income" in lname:
                    return random.randint(30000, 150000)
            return random.randint(1, 1000)

        if "REAL" in dtype or "FLOAT" in dtype or "DOUBLE" in dtype:
            # Smart decimal generation
            if col_name:
                lname = col_name.lower()
                if "salary" in lname or "income" in lname or "price" in lname:
                    return round(random.uniform(30000, 200000), 2)
                elif "percentage" in lname or "rate" in lname:
                    return round(random.uniform(0, 100), 2)
                elif "weight" in lname:
                    return round(random.uniform(50, 120), 2)
                elif "height" in lname:
                    return round(random.uniform(150, 200), 2)
            return round(random.uniform(1, 1000), 3)

        if "DATE" in dtype:
            # Generate dates within reasonable ranges
            if col_name:
                lname = col_name.lower()
                if "birth" in lname or "dob" in lname:
                    # Birth dates: 18-85 years ago
                    days_ago = random.randint(18*365, 85*365)
                    return (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
                elif "hire" in lname or "join" in lname or "start" in lname:
                    # Hire dates: within last 10 years
                    days_ago = random.randint(1, 10*365)
                    return (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
            return self.fake.date()

        if "BOOL" in dtype or "BOOLEAN" in dtype:
            return random.choice([0, 1])

        # Heuristics for TEXT columns
        if col_name:
            lname = col_name.lower()
            if "name" in lname and "user" in lname:
                return self.fake.name()
            if "name" in lname and ("first" in lname or "fname" in lname):
                return self.fake.first_name()
            if "name" in lname and ("last" in lname or "lname" in lname):
                return self.fake.last_name()
            if "name" in lname and ("company" in lname or "business" in lname):
                return self.fake.company()
            if "name" in lname:
                return self.fake.name()
            if "email" in lname:
                return self.fake.email()
            if "phone" in lname or "mobile" in lname or "tel" in lname:
                return self.fake.phone_number()
            if "address" in lname:
                return self.fake.address().replace("\\n", " ")
            if "street" in lname:
                return self.fake.street_address()
            if "city" in lname:
                return self.fake.city()
            if "state" in lname or "province" in lname:
                return self.fake.state()
            if "country" in lname:
                return self.fake.country()
            if "zip" in lname or "postal" in lname:
                return self.fake.zipcode()
            if "job" in lname or "position" in lname or "title" in lname:
                return self.fake.job()
            if "department" in lname or "dept" in lname:
                return random.choice(['Sales', 'Marketing', 'Engineering', 'HR', 'Finance', 'Operations', 'IT'])
            if "status" in lname:
                return random.choice(['Active', 'Inactive', 'Pending', 'Completed', 'Cancelled'])
            if "gender" in lname or "sex" in lname:
                return random.choice(['M', 'F', 'Male', 'Female'])
            if "desc" in lname or "text" in lname or "note" in lname or "comment" in lname:
                return self.fake.sentence(nb_words=random.randint(5, 15))
            if "url" in lname or "website" in lname:
                return self.fake.url()
            if "username" in lname or "user_name" in lname:
                return self.fake.user_name()
            if "product" in lname:
                return self.fake.word().capitalize() + " " + random.choice(['Pro', 'Plus', 'Premium', 'Basic', 'Standard'])
            if "category" in lname or "type" in lname:
                return random.choice(['Type A', 'Type B', 'Type C', 'Category 1', 'Category 2', 'Standard', 'Premium'])

        return self.fake.word()

    def insert_data(self, table_name: str, schema: Dict[str, str], num_rows: int) -> Tuple[bool, str]:
        """Insert fake data into a table
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
            num_rows: Number of rows to generate
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if not cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' does not exist!"

            # Generate data
            rows = []
            col_names = list(schema.keys())
            for i in range(num_rows):
                row = [self.generate_value(schema[c], c) for c in col_names]
                rows.append(tuple(row))

            # Insert data
            placeholders = ", ".join(["?"] * len(col_names))
            insert_sql = f'INSERT INTO "{table_name}" ({", ".join(["\\"" + c + "\\"" for c in col_names])}) VALUES ({placeholders});'
            cursor.executemany(insert_sql, rows)
            conn.commit()
            conn.close()

            return True, f"✅ Inserted {num_rows} rows into '{table_name}'!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def get_all_tables(self) -> List[str]:
        """Get list of all tables in database"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
            tables = [row[0] for row in cursor.fetchall()]
            conn.close()
            return tables
        except Exception as e:
            return []

    def get_table_info(self, table_name: str) -> pd.DataFrame:
        """Get information about a table"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(f"PRAGMA table_info('{table_name}')", conn)
            conn.close()
            return df
        except Exception as e:
            return pd.DataFrame()

print("✅ Dynamic Table Generator module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/table_generator.py', 'w') as f:
    f.write(table_generator_code)

print("✅ table_generator.py saved successfully!")

# ==========================================
# CELL 4: Create All Other Modules (NL-to-SQL, Data Quality, Query Generator)
# ==========================================

# (Keep existing modules - nl_to_sql.py, data_quality_checker.py, query_generator.py)
# Copy from your existing code...

nl_to_sql_code = '''import re
from typing import List, Tuple, Dict
import sqlite3
import pandas as pd
from datetime import datetime
import google.generativeai as genai

class SimpleNLtoSQL:
    """Gemini-powered NL to SQL converter for clinical trial queries"""

    def __init__(self, db_path, api_key=None):
        self.db_path = db_path
        self.api_key = api_key

        if self.api_key:
            try:
                genai.configure(api_key=self.api_key)
                self.model = genai.GenerativeModel("models/gemini-2.0-flash-exp")
                print("✅ Gemini API configured successfully!")
            except Exception as e:
                print(f"⚠️ Error configuring Gemini: {e}")
                self.model = None
        else:
            self.model = None

        self.schema_info = """
DATABASE SCHEMA FOR CLINICAL TRIAL:

1. patients table:
   - patient_id (TEXT PRIMARY KEY)
   - site_id (TEXT)
   - age (INTEGER)
   - gender (TEXT)
   - enrollment_date (DATE)
   - treatment_arm (TEXT)
   - status (TEXT)

2. adverse_events table:
   - event_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - event_term (TEXT)
   - severity (TEXT)
   - event_date (DATE)
   - resolved (TEXT)

3. lab_results table:
   - lab_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - test_name (TEXT)
   - test_value (REAL)
   - unit (TEXT)
   - test_date (DATE)
   - normal_range_low (REAL)
   - normal_range_high (REAL)

4. visits table:
   - visit_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - visit_number (INTEGER)
   - visit_date (DATE)
   - scheduled_date (DATE)
   - visit_type (TEXT)
   - completed (TEXT)
"""

    def parse_query(self, nl_query: str) -> Tuple[str, str, str]:
        """Convert natural language to SQL using Gemini API"""
        nl_query = nl_query.strip()

        if not nl_query:
            return None, "error", "Please enter a query."

        if not self.model:
            return None, "error", "⚠️ Gemini API not configured. Please provide API key in the app."

        try:
            prompt = f"""You are a SQL expert for a clinical trial database.

{self.schema_info}

IMPORTANT RULES:
1. Generate ONLY valid SQLite syntax
2. Return ONLY the SQL query, no explanations or markdown
3. Use proper table and column names from the schema above
4. Always add LIMIT 100 to prevent large result sets
5. For "outliers" or "out of range" labs, use: test_value < normal_range_low OR test_value > normal_range_high

USER QUERY: "{nl_query}"

Generate the SQL query:"""

            response = self.model.generate_content(prompt)

            if not response or not response.text:
                return None, "error", "No response from Gemini API"

            sql_query = self._extract_sql(response.text)

            if not sql_query:
                return None, "error", "Could not extract valid SQL from response"

            is_valid, validation_msg = self._validate_sql(sql_query)

            if not is_valid:
                return None, "error", f"Invalid SQL: {validation_msg}"

            explanation = f"Natural Language: {nl_query}\\n\\nGenerated SQL Query:\\n{sql_query}"

            return sql_query, "success", explanation

        except Exception as e:
            return None, "error", f"Error generating SQL: {str(e)}"

    def _extract_sql(self, response_text: str) -> str:
        """Extract SQL query from Gemini response"""
        response_text = response_text.strip()

        if response_text.startswith('```'):
            lines = response_text.split('\\n')
            lines = lines[1:]
            if lines and lines[-1].strip() == '```':
                lines = lines[:-1]
            response_text = '\\n'.join(lines)

        sql_query = response_text.strip()

        if not sql_query.upper().startswith('SELECT'):
            match = re.search(r'(SELECT\\s+.+)', sql_query, re.IGNORECASE | re.DOTALL)
            if match:
                sql_query = match.group(1)

        return sql_query

    def _validate_sql(self, sql_query: str) -> Tuple[bool, str]:
        """Validate SQL query"""
        if not sql_query.upper().strip().startswith('SELECT'):
            return False, "Only SELECT queries are allowed"

        dangerous_keywords = ['DROP', 'DELETE', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE']
        upper_query = sql_query.upper()
        for keyword in dangerous_keywords:
            if keyword in upper_query:
                return False, f"Query contains forbidden keyword: {keyword}"

        return True, "Valid"

    def execute_query(self, sql_query: str) -> Tuple:
        """Execute SQL query and return results"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(sql_query, conn)
            conn.close()
            return df, "success"
        except Exception as e:
            return None, f"Error executing query: {str(e)}"

    def get_example_queries(self) -> List[str]:
        """Return list of example queries"""
        return [
            "Show all patients",
            "How many patients per site?",
            "List patients with severe adverse events",
            "Show patients older than 65",
            "Find lab results with outliers",
            "Count adverse events by severity"
        ]

print("✅ Gemini-powered NL-to-SQL module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/nl_to_sql.py', 'w') as f:
    f.write(nl_to_sql_code)

# (Add data_quality_checker.py and query_generator.py - keeping your existing code)

print("✅ All modules created successfully!")

# ==========================================
# CELL 5: Create Enhanced Gradio App with Table Generator
# ==========================================

app_code = '''import gradio as gr
import sqlite3
import pandas as pd
import plotly.express as px
from datetime import datetime
import sys
import os

sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from nl_to_sql import SimpleNLtoSQL
from table_generator import DynamicTableGenerator

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

nl_sql = None
table_gen = DynamicTableGenerator(DB_PATH)

def configure_api(api_key):
    global nl_sql
    if not api_key or api_key.strip() == "":
        return "⚠️ Please enter your Gemini API key", False

    try:
        nl_sql = SimpleNLtoSQL(DB_PATH, api_key=api_key.strip())
        return "✅ Gemini API configured! You can now use the query interface.", True
    except Exception as e:
        return f"❌ Error: {str(e)}", False

def execute_nl_query(user_query, confirmation):
    global nl_sql

    if not nl_sql:
        return "⚠️ Please configure Gemini API first!", None, "", gr.update(visible=False)

    if not user_query or user_query.strip() == "":
        return "Please enter a query.", None, "", gr.update(visible=False)

    try:
        sql_query, status, explanation = nl_sql.parse_query(user_query)

        if status == "success" and sql_query:
            if not confirmation:
                return (
                    f"🔍 Generated SQL Query:\\n\\n{sql_query}\\n\\n" +
                    f"💡 Explanation:\\n{explanation}\\n\\n" +
                    "⚠️ Please review and check the confirmation box to execute.",
                    None,
                    sql_query,
                    gr.update(visible=True)
                )

            df_result, exec_status = nl_sql.execute_query(sql_query)

            if exec_status == "success":
                result_text = f"✅ Query executed!\\n\\n📊 Found {len(df_result)} results"
                return result_text, df_result, sql_query, gr.update(visible=False)
            else:
                return f"❌ Error: {exec_status}", None, sql_query, gr.update(visible=False)
        else:
            return f"⚠️ {explanation}", None, sql_query or "", gr.update(visible=False)

    except Exception as e:
        return f"❌ Error: {str(e)}", None, "", gr.update(visible=False)

def create_custom_table(table_name, columns_text, num_rows):
    """Create a new custom table"""
    if not table_name or not columns_text:
        return "⚠️ Please provide table name and columns!", None

    # Parse columns (format: col1:TEXT, col2:INTEGER, col3:DATE)
    try:
        schema = {}
        for line in columns_text.strip().split('\n'):
            if ':' in line:
                col, dtype = line.split(':')
                schema[col.strip()] = dtype.strip()

        if not schema:
            return "⚠️ Invalid column format! Use: column_name:DATA_TYPE (one per line)", None

        # Show what data will be generated
        preview_info = "📋 Table Schema Preview:\n\n"
        for col, dtype in schema.items():
            data_hint = ""
            lname = col.lower()
            if "age" in lname:
                data_hint = " → Will generate ages 18-85"
            elif "email" in lname:
                data_hint = " → Will generate valid emails"
            elif "phone" in lname or "mobile" in lname:
                data_hint = " → Will generate phone numbers"
            elif "salary" in lname or "price" in lname:
                data_hint = " → Will generate $30K-$200K"
            elif "name" in lname:
                data_hint = " → Will generate realistic names"
            elif "date" in lname:
                if "birth" in lname or "dob" in lname:
                    data_hint = " → Will generate birth dates (18-85 years old)"
                elif "hire" in lname or "join" in lname:
                    data_hint = " → Will generate hire dates (last 10 years)"
                else:
                    data_hint = " → Will generate random dates"
            elif "department" in lname:
                data_hint = " → Will generate: Sales, Engineering, HR, etc."
            elif "status" in lname:
                data_hint = " → Will generate: Active, Inactive, Pending, etc."
            elif "gender" in lname:
                data_hint = " → Will generate: M/F or Male/Female"

            preview_info += f"  • {col} ({dtype}){data_hint}\n"

        preview_info += f"\n📊 Creating {num_rows} rows...\n\n"

        # Create table
        success, msg = table_gen.create_table_from_schema(table_name, schema)
        if not success:
            return msg, None

        # Insert data
        success, data_msg = table_gen.insert_data(table_name, schema, int(num_rows))

        # Show preview
        conn = sqlite3.connect(DB_PATH)
        df = pd.read_sql_query(f'SELECT * FROM "{table_name}" LIMIT 10', conn)
        conn.close()

        final_msg = preview_info + msg + "\n" + data_msg + "\n\n✅ Table created successfully with realistic data!\n\n💡 Now you can query it using natural language!"

        return final_msg, df

    except Exception as e:
        return f"❌ Error: {str(e)}", None

def list_all_tables():
    """List all tables in database"""
    tables = table_gen.get_all_tables()
    return "\\n".join([f"• {t}" for t in tables])

with gr.Blocks(title="Clinical Trial Data Management", theme=gr.themes.Soft()) as app:

    gr.Markdown("""
    # 🏥 Clinical Trial Data Management System
    ### AI-Powered with Google Gemini + Dynamic Table Generator
    **Version 3.0**
    """)

    with gr.Tab("🔑 API Configuration"):
        gr.Markdown("### Configure Gemini API")

        api_key_input = gr.Textbox(
            label="🔑 Enter Gemini API Key",
            type="password",
            placeholder="Get from https://aistudio.google.com/apikey"
        )
        api_config_btn = gr.Button("⚙️ Configure API", variant="primary")
        api_status = gr.Textbox(label="API Status", lines=2)
        api_configured = gr.State(False)

        api_config_btn.click(
            fn=configure_api,
            inputs=[api_key_input],
            outputs=[api_status, api_configured]
        )

    with gr.Tab("🔍 Query Interface"):
        gr.Markdown("### Ask questions in plain English - Powered by Gemini 2.0")

        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Your question:",
                    placeholder="e.g., Show patients with severe adverse events",
                    lines=2
                )

                confirmation_checkbox = gr.Checkbox(
                    label="✅ I reviewed the SQL and want to execute it",
                    value=False,
                    visible=False
                )

                with gr.Row():
                    query_btn = gr.Button("🚀 Generate SQL", variant="primary")
                    clear_btn = gr.Button("🗑️ Clear")

        query_output = gr.Textbox(label="Results", lines=8)
        sql_output = gr.Code(label="Generated SQL", language="sql")
        results_df = gr.Dataframe(label="Data", wrap=True)

        query_btn.click(
            fn=execute_nl_query,
            inputs=[query_input, confirmation_checkbox],
            outputs=[query_output, results_df, sql_output, confirmation_checkbox]
        )

        clear_btn.click(
            fn=lambda: ("", None, "", False, gr.update(visible=False)),
            outputs=[query_output, results_df, sql_output, confirmation_checkbox, confirmation_checkbox]
        )

    with gr.Tab("🆕 Create Custom Tables"):
        gr.Markdown("""
        ### 🎨 Dynamic Table Generator
        Create your own custom tables with automatic fake data generation!
        """)

        with gr.Row():
            with gr.Column():
                new_table_name = gr.Textbox(
                    label="📋 Table Name",
                    placeholder="e.g., custom_data"
                )

                columns_input = gr.Textbox(
                    label="📝 Columns (one per line)",
                    placeholder="column1:TEXT\\ncolumn2:INTEGER\\ncolumn3:DATE\\ncolumn4:REAL",
                    lines=8
                )

                gr.Markdown("""
                **Supported Data Types:**
                - TEXT (for names, emails, addresses, etc.)
                - INTEGER (for numbers, ages, IDs)
                - REAL (for decimals, prices, salaries)
                - DATE (for dates)
                - BOOLEAN (for yes/no values)

                **Smart Column Detection Examples:**
                - Columns with "age" → generates ages 18-85
                - Columns with "email" → generates valid emails
                - Columns with "phone/mobile" → generates phone numbers
                - Columns with "salary/price" → generates realistic amounts
                - Columns with "date/birth/dob" → generates birth dates
                - Columns with "hire/join" → generates employment dates
                - Columns with "department" → generates dept names
                - Columns with "status" → generates status values
                - Columns with "gender" → generates M/F values

                **💡 Pro Tips:**
                1. Use descriptive column names for better data
                2. Column name "user_age" will generate realistic ages
                3. Column name "emp_salary" will generate salary ranges
                4. Column name "birth_date" will generate birth dates
                """)

                gr.Markdown("""
                ---
                **📝 Quick Examples:**

                **Employee Table:**
                ```
                emp_name:TEXT
                emp_email:TEXT
                emp_age:INTEGER
                emp_salary:REAL
                department:TEXT
                hire_date:DATE
                status:TEXT
                ```

                **Product Table:**
                ```
                product_name:TEXT
                product_price:REAL
                category:TEXT
                stock_quantity:INTEGER
                ```

                **Customer Table:**
                ```
                customer_name:TEXT
                customer_email:TEXT
                phone_number:TEXT
                address:TEXT
                city:TEXT
                ```
                """)

                num_rows_input = gr.Slider(
                    minimum=1,
                    maximum=1000,
                    value=100,
                    step=1,
                    label="📊 Number of Rows to Generate"
                )

                create_table_btn = gr.Button("🎯 Create Table & Generate Data", variant="primary")

            with gr.Column():
                gr.Markdown("### 📚 Existing Tables")
                tables_list = gr.Textbox(
                    label="Current Tables in Database",
                    lines=15,
                    interactive=False
                )
                refresh_tables_btn = gr.Button("🔄 Refresh List")

        create_output = gr.Textbox(label="Status", lines=5)
        preview_df = gr.Dataframe(label="Preview (First 10 Rows)", wrap=True)

        create_table_btn.click(
            fn=create_custom_table,
            inputs=[new_table_name, columns_input, num_rows_input],
            outputs=[create_output, preview_df]
        )

        refresh_tables_btn.click(
            fn=list_all_tables,
            outputs=[tables_list]
        )

        # Auto-load tables on tab load
        app.load(fn=list_all_tables, outputs=[tables_list])

    with gr.Tab("📊 Dashboard"):
        gr.Markdown("### Analytics & Metrics")

        def create_dashboard():
            try:
                conn = sqlite3.connect(DB_PATH)

                # Get stats
                stats = {
                    'total_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients", conn).iloc[0]['count'],
                    'active_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients WHERE status = 'Active'", conn).iloc[0]['count'],
                    'total_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events", conn).iloc[0]['count'],
                    'severe_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events WHERE severity = 'Severe'", conn).iloc[0]['count'],
                }

                summary = f"""
CLINICAL TRIAL DASHBOARD
Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

KEY METRICS:

👥 Patients:
   • Total: {stats['total_patients']}
   • Active: {stats['active_patients']} ({stats['active_patients']/stats['total_patients']*100:.1f}%)

⚠️ Safety:
   • Total AEs: {stats['total_aes']}
   • Severe AEs: {stats['severe_aes']}
"""

                site_dist = pd.read_sql_query("SELECT site_id, COUNT(*) as count FROM patients GROUP BY site_id", conn)
                fig1 = px.bar(site_dist, x='site_id', y='count', title="Patients by Site")

                treatment_dist = pd.read_sql_query("SELECT treatment_arm, COUNT(*) as count FROM patients GROUP BY treatment_arm", conn)
                fig2 = px.pie(treatment_dist, values='count', names='treatment_arm', title="Treatment Arms")

                conn.close()

                return summary, fig1, fig2

            except Exception as e:
                return f"❌ Error: {str(e)}", None, None

        refresh_btn = gr.Button("🔄 Refresh", variant="primary")

        dashboard_summary = gr.Textbox(label="Metrics", lines=20)

        with gr.Row():
            chart1 = gr.Plot(label="Patients by Site")
            chart2 = gr.Plot(label="Treatment Arms")

        refresh_btn.click(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

        app.load(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

    gr.Markdown("""
    ---
    **v3.0** | Powered by Google Gemini | Data in Google Drive | Dynamic Table Generator

    💾 Location: `/content/drive/MyDrive/ClinicalTrialApp/`

    🎯 **New Feature**: Create custom tables with automatic fake data generation!
    """)

print("✅ Enhanced Gradio app created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/app_gradio.py', 'w') as f:
    f.write(app_code)

print("✅ app_gradio.py saved successfully!")

# ==========================================
# CELL 6: Instructions
# ==========================================

print("""
╔═══════════════════════════════════════════════════════════╗
║    🎉 INTEGRATED APP WITH TABLE GENERATOR READY!          ║
╚═══════════════════════════════════════════════════════════╝

📝 HOW TO GET YOUR GEMINI API KEY:

1. 🌐 Go to: https://aistudio.google.com/apikey
2. 🔑 Click "Create API Key" button
3. 📋 Copy your API key
4. ✅ Paste it in the app when it launches
5. ⚙️ Click "Configure API" button

🎯 NEW FEATURES:
   ✅ AI SQL generation (Gemini 2.0)
   ✅ Query confirmation (safety first!)
   ✅ Interactive dashboard
   ✨ Dynamic table creation with fake data!
   ✨ Custom schema definition
   ✨ Automatic data type detection
   ✨ Real-time table listing

💾 DATA: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this notebook running!
""")

# ==========================================
# CELL 7: Launch the App
# ==========================================

import os
os.chdir('/content/drive/MyDrive/ClinicalTrialApp')

import sys
sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from app_gradio import app

app.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("""
╔═══════════════════════════════════════════════════════════╗
║  🎉 APP IS RUNNING!                                       ║
╚═══════════════════════════════════════════════════════════╝

🔑 SETUP:
   1. Click the public URL above
   2. Enter your Gemini API key in "API Configuration" tab
   3. Click "Configure API"
   4. Start using all features!

✨ TABS AVAILABLE:
   1️⃣ API Configuration - Setup Gemini
   2️⃣ Query Interface - Ask questions in plain English
   3️⃣ Create Custom Tables - NEW! Generate tables dynamically
   4️⃣ Dashboard - View analytics

🎨 EXAMPLE: CREATE CUSTOM TABLE
   Table Name: employee_data
   Columns:
   emp_name:TEXT
   emp_email:TEXT
   emp_age:INTEGER
   emp_salary:REAL
   department:TEXT
   hire_date:DATE
   status:TEXT

   Set rows to 100 → Click "Create Table & Generate Data"

   This will create realistic data:
   - emp_age: Ages between 18-85
   - emp_salary: Salaries $30,000-$200,000
   - hire_date: Employment dates within last 10 years
   - department: Sales, Engineering, HR, etc.
   - status: Active, Inactive, etc.

   Then query it: "show me employees older than 40"

💾 DATA SAVED IN: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this cell running to use the app!

🔄 To restart: Run Cell 7 again
""")

Mounted at /content/drive
✅ Google Drive mounted successfully!
📁 Project directory: /content/drive/MyDrive/ClinicalTrialApp
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 31.4 MB/s eta 0:00:00
✅ All packages installed successfully!
✅ Clinical trial database created successfully!
📁 Database location: /content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db
   - 50 patients
   - 85 adverse events
   - 378 lab results
   - 200 visits

📊 Sample Patients:


,patient_id,site_id,age,gender,enrollment_date,treatment_arm,status
0,PT0001,SITE005,46,M,2025-03-08,Treatment B,Completed
1,PT0002,SITE004,26,M,2025-06-01,Treatment A,Completed
2,PT0003,SITE005,68,F,2025-08-14,Treatment B,Withdrawn
3,PT0004,SITE003,43,F,2025-08-16,Placebo,Completed
4,PT0005,SITE005,66,F,2024-11-29,Treatment A,Completed



💾 Database is now permanently stored in your Google Drive!
✅ table_generator.py saved successfully!
✅ All modules created successfully!
✅ app_gradio.py saved successfully!

╔═══════════════════════════════════════════════════════════╗
║    🎉 INTEGRATED APP WITH TABLE GENERATOR READY!          ║
╚═══════════════════════════════════════════════════════════╝

📝 HOW TO GET YOUR GEMINI API KEY:

1. 🌐 Go to: https://aistudio.google.com/apikey
2. 🔑 Click "Create API Key" button
3. 📋 Copy your API key
4. ✅ Paste it in the app when it launches
5. ⚙️ Click "Configure API" button

🎯 NEW FEATURES:
   ✅ AI SQL generation (Gemini 2.0)
   ✅ Query confirmation (safety first!)
   ✅ Interactive dashboard
   ✨ Dynamic table creation with fake data!
   ✨ Custom schema definition
   ✨ Automatic data type detection
   ✨ Real-time table listing

💾 DATA: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this notebook running!



SyntaxError: unterminated string literal (detected at line 74) (app_gradio.py, line 74)

In [ ]:
# ==========================================
# INTEGRATED CLINICAL TRIAL APP WITH TABLE GENERATOR
# Run this complete notebook in Google Colab
# ==========================================

# CELL 1: Setup and Mount Google Drive
# ==========================================

from google.colab import drive
drive.mount('/content/drive')

import os
project_dir = '/content/drive/MyDrive/ClinicalTrialApp'
os.makedirs(project_dir, exist_ok=True)
os.chdir(project_dir)

print(f"✅ Google Drive mounted successfully!")
print(f"📁 Project directory: {project_dir}")

!pip install -q pandas numpy plotly sqlalchemy openpyxl gradio google-generativeai faker

print("✅ All packages installed successfully!")

# ==========================================
# CELL 2: Create Database with Sample Data
# ==========================================

import sqlite3
import pandas as pd
import random
from datetime import datetime, timedelta
import numpy as np

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

def create_clinical_database():
    """Create clinical trial database with sample data"""
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    # Drop existing tables if they exist
    cursor.execute('DROP TABLE IF EXISTS patients')
    cursor.execute('DROP TABLE IF EXISTS adverse_events')
    cursor.execute('DROP TABLE IF EXISTS lab_results')
    cursor.execute('DROP TABLE IF EXISTS visits')
    cursor.execute('DROP TABLE IF EXISTS query_log')

    # Create Patients table
    cursor.execute('''
    CREATE TABLE patients (
        patient_id TEXT PRIMARY KEY,
        site_id TEXT,
        age INTEGER,
        gender TEXT,
        enrollment_date DATE,
        treatment_arm TEXT,
        status TEXT
    )
    ''')

    # Create Adverse Events table
    cursor.execute('''
    CREATE TABLE adverse_events (
        event_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        event_term TEXT,
        severity TEXT,
        event_date DATE,
        resolved TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Lab Results table
    cursor.execute('''
    CREATE TABLE lab_results (
        lab_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        test_name TEXT,
        test_value REAL,
        unit TEXT,
        test_date DATE,
        normal_range_low REAL,
        normal_range_high REAL,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Visits table
    cursor.execute('''
    CREATE TABLE visits (
        visit_id INTEGER PRIMARY KEY AUTOINCREMENT,
        patient_id TEXT,
        visit_number INTEGER,
        visit_date DATE,
        scheduled_date DATE,
        visit_type TEXT,
        completed TEXT,
        FOREIGN KEY (patient_id) REFERENCES patients(patient_id)
    )
    ''')

    # Create Query Log table
    cursor.execute('''
    CREATE TABLE query_log (
        log_id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp DATETIME,
        natural_language_query TEXT,
        generated_sql TEXT,
        execution_status TEXT,
        results_count INTEGER
    )
    ''')

    # Insert sample patients (50 patients)
    sites = ['SITE001', 'SITE002', 'SITE003', 'SITE004', 'SITE005']
    genders = ['M', 'F']
    treatment_arms = ['Treatment A', 'Treatment B', 'Placebo']
    statuses = ['Active', 'Completed', 'Withdrawn']

    patients_data = []
    for i in range(1, 51):
        patient_id = f'PT{i:04d}'
        site_id = random.choice(sites)
        # More realistic age distribution with varied range
        age_range = random.choice([
            (18, 30),   # Young adults
            (31, 45),   # Adults
            (46, 60),   # Middle-aged
            (61, 75),   # Senior
            (76, 85)    # Elderly
        ])
        age = random.randint(age_range[0], age_range[1])
        gender = random.choice(genders)
        enrollment_date = (datetime.now() - timedelta(days=random.randint(30, 365))).strftime('%Y-%m-%d')
        treatment_arm = random.choice(treatment_arms)
        status = random.choice(statuses)
        patients_data.append((patient_id, site_id, age, gender, enrollment_date, treatment_arm, status))

    cursor.executemany('INSERT INTO patients VALUES (?, ?, ?, ?, ?, ?, ?)', patients_data)

    # Insert adverse events
    adverse_events_data = []
    event_terms = ['Headache', 'Nausea', 'Fatigue', 'Dizziness', 'Rash', 'Fever', 'Insomnia', 'Chest Pain']
    severities = ['Mild', 'Moderate', 'Severe', None]

    for patient in patients_data[:35]:
        num_events = random.randint(1, 4)
        for j in range(num_events):
            patient_id = patient[0]
            event_term = random.choice(event_terms)
            severity = random.choice(severities)
            event_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            resolved = random.choice(['Yes', 'No', None])
            adverse_events_data.append((patient_id, event_term, severity, event_date, resolved))

    cursor.executemany('INSERT INTO adverse_events (patient_id, event_term, severity, event_date, resolved) VALUES (?, ?, ?, ?, ?)',
                       adverse_events_data)

    # Insert lab results
    lab_tests = [
        ('Hemoglobin', 'g/dL', 12.0, 16.0),
        ('White Blood Cell Count', '10^9/L', 4.0, 11.0),
        ('Platelet Count', '10^9/L', 150, 400),
        ('ALT', 'U/L', 7, 56),
        ('AST', 'U/L', 10, 40),
        ('Creatinine', 'mg/dL', 0.7, 1.3),
        ('Glucose', 'mg/dL', 70, 100),
        ('Sodium', 'mmol/L', 135, 145),
        ('Potassium', 'mmol/L', 3.5, 5.0)
    ]

    lab_results_data = []
    for patient in patients_data:
        num_labs = random.randint(5, 10)
        for _ in range(num_labs):
            patient_id = patient[0]
            test = random.choice(lab_tests)
            test_name = test[0]
            unit = test[1]
            normal_low = test[2]
            normal_high = test[3]

            rand_val = random.random()
            if rand_val < 0.75:
                test_value = round(random.uniform(normal_low, normal_high), 2)
            elif rand_val < 0.95:
                test_value = round(random.uniform(normal_high, normal_high * 1.5), 2)
            else:
                test_value = round(random.uniform(normal_low * 0.5, normal_low), 2)

            test_date = (datetime.now() - timedelta(days=random.randint(1, 180))).strftime('%Y-%m-%d')
            lab_results_data.append((patient_id, test_name, test_value, unit, test_date, normal_low, normal_high))

    cursor.executemany('INSERT INTO lab_results (patient_id, test_name, test_value, unit, test_date, normal_range_low, normal_range_high) VALUES (?, ?, ?, ?, ?, ?, ?)',
                       lab_results_data)

    # Insert visits
    visits_data = []
    for patient in patients_data:
        for visit_num in range(1, 5):
            patient_id = patient[0]
            scheduled_date = datetime.strptime(patient[4], '%Y-%m-%d') + timedelta(days=30*visit_num)

            if random.random() < 0.65:
                visit_date = scheduled_date + timedelta(days=random.randint(-2, 2))
            else:
                visit_date = scheduled_date + timedelta(days=random.randint(8, 30))

            visit_type = f'Visit {visit_num}'
            completed = random.choice(['Yes', 'No', None])
            visits_data.append((patient_id, visit_num, visit_date.strftime('%Y-%m-%d'),
                              scheduled_date.strftime('%Y-%m-%d'), visit_type, completed))

    cursor.executemany('INSERT INTO visits (patient_id, visit_number, visit_date, scheduled_date, visit_type, completed) VALUES (?, ?, ?, ?, ?, ?)',
                       visits_data)

    conn.commit()
    conn.close()

    print("✅ Clinical trial database created successfully!")
    print(f"📁 Database location: {DB_PATH}")
    print(f"   - {len(patients_data)} patients")
    print(f"   - {len(adverse_events_data)} adverse events")
    print(f"   - {len(lab_results_data)} lab results")
    print(f"   - {len(visits_data)} visits")

    return True

create_clinical_database()

conn = sqlite3.connect(DB_PATH)
print("\n📊 Sample Patients:")
display(pd.read_sql_query("SELECT * FROM patients LIMIT 5", conn))
conn.close()

print("\n💾 Database is now permanently stored in your Google Drive!")

# ==========================================
# CELL 3: Create Dynamic Table Generator Module
# ==========================================

table_generator_code = '''from faker import Faker
import pandas as pd
import sqlite3
import random
from typing import Dict, List, Tuple

class DynamicTableGenerator:
    """Generate custom tables dynamically based on user specifications"""

    def __init__(self, db_path):
        self.db_path = db_path
        self.fake = Faker()
        Faker.seed(0)
        random.seed(0)

    def create_table_from_schema(self, table_name: str, schema: Dict[str, str]) -> Tuple[bool, str]:
        """Create a new table in the database
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table already exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' already exists!"

            # Build column definitions
            cols_def = []
            for col, dtype in schema.items():
                dtype_sql = "INTEGER" if dtype.upper() == "BOOLEAN" else dtype.upper()
                cols_def.append(f'"{col}" {dtype_sql}')

            create_sql = f'CREATE TABLE "{table_name}" ({", ".join(cols_def)});'
            cursor.execute(create_sql)
            conn.commit()
            conn.close()

            return True, f"✅ Table '{table_name}' created successfully!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def generate_value(self, dtype: str, col_name: str = None):
        """Generate fake data based on data type and column name"""
        dtype = dtype.upper()

        if "INT" in dtype:
            # Smart integer generation based on column name
            if col_name:
                lname = col_name.lower()
                if "age" in lname:
                    return random.randint(18, 85)
                elif "year" in lname:
                    return random.randint(1950, 2025)
                elif "quantity" in lname or "count" in lname:
                    return random.randint(1, 100)
                elif "id" in lname:
                    return random.randint(1000, 9999)
                elif "salary" in lname or "income" in lname:
                    return random.randint(30000, 150000)
            return random.randint(1, 1000)

        if "REAL" in dtype or "FLOAT" in dtype or "DOUBLE" in dtype:
            # Smart decimal generation
            if col_name:
                lname = col_name.lower()
                if "salary" in lname or "income" in lname or "price" in lname:
                    return round(random.uniform(30000, 200000), 2)
                elif "percentage" in lname or "rate" in lname:
                    return round(random.uniform(0, 100), 2)
                elif "weight" in lname:
                    return round(random.uniform(50, 120), 2)
                elif "height" in lname:
                    return round(random.uniform(150, 200), 2)
            return round(random.uniform(1, 1000), 3)

        if "DATE" in dtype:
            # Generate dates within reasonable ranges
            if col_name:
                lname = col_name.lower()
                if "birth" in lname or "dob" in lname:
                    # Birth dates: 18-85 years ago
                    days_ago = random.randint(18*365, 85*365)
                    return (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
                elif "hire" in lname or "join" in lname or "start" in lname:
                    # Hire dates: within last 10 years
                    days_ago = random.randint(1, 10*365)
                    return (datetime.now() - timedelta(days=days_ago)).strftime('%Y-%m-%d')
            return self.fake.date()

        if "BOOL" in dtype or "BOOLEAN" in dtype:
            return random.choice([0, 1])

        # Heuristics for TEXT columns
        if col_name:
            lname = col_name.lower()
            if "name" in lname and "user" in lname:
                return self.fake.name()
            if "name" in lname and ("first" in lname or "fname" in lname):
                return self.fake.first_name()
            if "name" in lname and ("last" in lname or "lname" in lname):
                return self.fake.last_name()
            if "name" in lname and ("company" in lname or "business" in lname):
                return self.fake.company()
            if "name" in lname:
                return self.fake.name()
            if "email" in lname:
                return self.fake.email()
            if "phone" in lname or "mobile" in lname or "tel" in lname:
                return self.fake.phone_number()
            if "address" in lname:
                return self.fake.address().replace("\\n", " ")
            if "street" in lname:
                return self.fake.street_address()
            if "city" in lname:
                return self.fake.city()
            if "state" in lname or "province" in lname:
                return self.fake.state()
            if "country" in lname:
                return self.fake.country()
            if "zip" in lname or "postal" in lname:
                return self.fake.zipcode()
            if "job" in lname or "position" in lname or "title" in lname:
                return self.fake.job()
            if "department" in lname or "dept" in lname:
                return random.choice(['Sales', 'Marketing', 'Engineering', 'HR', 'Finance', 'Operations', 'IT'])
            if "status" in lname:
                return random.choice(['Active', 'Inactive', 'Pending', 'Completed', 'Cancelled'])
            if "gender" in lname or "sex" in lname:
                return random.choice(['M', 'F', 'Male', 'Female'])
            if "desc" in lname or "text" in lname or "note" in lname or "comment" in lname:
                return self.fake.sentence(nb_words=random.randint(5, 15))
            if "url" in lname or "website" in lname:
                return self.fake.url()
            if "username" in lname or "user_name" in lname:
                return self.fake.user_name()
            if "product" in lname:
                return self.fake.word().capitalize() + " " + random.choice(['Pro', 'Plus', 'Premium', 'Basic', 'Standard'])
            if "category" in lname or "type" in lname:
                return random.choice(['Type A', 'Type B', 'Type C', 'Category 1', 'Category 2', 'Standard', 'Premium'])

        return self.fake.word()

    def insert_data(self, table_name: str, schema: Dict[str, str], num_rows: int) -> Tuple[bool, str]:
        """Insert fake data into a table
        Args:
            table_name: Name of the table
            schema: Dict with column_name: data_type pairs
            num_rows: Number of rows to generate
        Returns:
            (success, message)
        """
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()

            # Check if table exists
            cursor.execute(f"SELECT name FROM sqlite_master WHERE type='table' AND name='{table_name}'")
            if not cursor.fetchone():
                conn.close()
                return False, f"❌ Table '{table_name}' does not exist!"

            # Generate data
            rows = []
            col_names = list(schema.keys())
            for i in range(num_rows):
                row = [self.generate_value(schema[c], c) for c in col_names]
                rows.append(tuple(row))

            # Insert data
            placeholders = ", ".join(["?"] * len(col_names))
            insert_sql = f'INSERT INTO "{table_name}" ({", ".join(["\\"" + c + "\\"" for c in col_names])}) VALUES ({placeholders});'
            cursor.executemany(insert_sql, rows)
            conn.commit()
            conn.close()

            return True, f"✅ Inserted {num_rows} rows into '{table_name}'!"

        except Exception as e:
            return False, f"❌ Error: {str(e)}"

    def get_all_tables(self) -> List[str]:
        """Get list of all tables in database"""
        try:
            conn = sqlite3.connect(self.db_path)
            cursor = conn.cursor()
            cursor.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name")
            tables = [row[0] for row in cursor.fetchall()]
            conn.close()
            return tables
        except Exception as e:
            return []

    def get_table_info(self, table_name: str) -> pd.DataFrame:
        """Get information about a table"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(f"PRAGMA table_info('{table_name}')", conn)
            conn.close()
            return df
        except Exception as e:
            return pd.DataFrame()

print("✅ Dynamic Table Generator module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/table_generator.py', 'w') as f:
    f.write(table_generator_code)

print("✅ table_generator.py saved successfully!")

# ==========================================
# CELL 4: Create All Other Modules (NL-to-SQL, Data Quality, Query Generator)
# ==========================================

# (Keep existing modules - nl_to_sql.py, data_quality_checker.py, query_generator.py)
# Copy from your existing code...

nl_to_sql_code = '''import re
from typing import List, Tuple, Dict
import sqlite3
import pandas as pd
from datetime import datetime
import google.generativeai as genai

class SimpleNLtoSQL:
    """Gemini-powered NL to SQL converter for clinical trial queries"""

    def __init__(self, db_path, api_key=None):
        self.db_path = db_path
        self.api_key = api_key

        if self.api_key:
            try:
                genai.configure(api_key=self.api_key)
                self.model = genai.GenerativeModel("models/gemini-2.0-flash-exp")
                print("✅ Gemini API configured successfully!")
            except Exception as e:
                print(f"⚠️ Error configuring Gemini: {e}")
                self.model = None
        else:
            self.model = None

        self.schema_info = """
DATABASE SCHEMA FOR CLINICAL TRIAL:

1. patients table:
   - patient_id (TEXT PRIMARY KEY)
   - site_id (TEXT)
   - age (INTEGER)
   - gender (TEXT)
   - enrollment_date (DATE)
   - treatment_arm (TEXT)
   - status (TEXT)

2. adverse_events table:
   - event_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - event_term (TEXT)
   - severity (TEXT)
   - event_date (DATE)
   - resolved (TEXT)

3. lab_results table:
   - lab_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - test_name (TEXT)
   - test_value (REAL)
   - unit (TEXT)
   - test_date (DATE)
   - normal_range_low (REAL)
   - normal_range_high (REAL)

4. visits table:
   - visit_id (INTEGER PRIMARY KEY)
   - patient_id (TEXT, FOREIGN KEY)
   - visit_number (INTEGER)
   - visit_date (DATE)
   - scheduled_date (DATE)
   - visit_type (TEXT)
   - completed (TEXT)
"""

    def parse_query(self, nl_query: str) -> Tuple[str, str, str]:
        """Convert natural language to SQL using Gemini API"""
        nl_query = nl_query.strip()

        if not nl_query:
            return None, "error", "Please enter a query."

        if not self.model:
            return None, "error", "⚠️ Gemini API not configured. Please provide API key in the app."

        try:
            prompt = f"""You are a SQL expert for a clinical trial database.

{self.schema_info}

IMPORTANT RULES:
1. Generate ONLY valid SQLite syntax
2. Return ONLY the SQL query, no explanations or markdown
3. Use proper table and column names from the schema above
4. Always add LIMIT 100 to prevent large result sets
5. For "outliers" or "out of range" labs, use: test_value < normal_range_low OR test_value > normal_range_high

USER QUERY: "{nl_query}"

Generate the SQL query:"""

            response = self.model.generate_content(prompt)

            if not response or not response.text:
                return None, "error", "No response from Gemini API"

            sql_query = self._extract_sql(response.text)

            if not sql_query:
                return None, "error", "Could not extract valid SQL from response"

            is_valid, validation_msg = self._validate_sql(sql_query)

            if not is_valid:
                return None, "error", f"Invalid SQL: {validation_msg}"

            explanation = f"Natural Language: {nl_query}\\n\\nGenerated SQL Query:\\n{sql_query}"

            return sql_query, "success", explanation

        except Exception as e:
            return None, "error", f"Error generating SQL: {str(e)}"

    def _extract_sql(self, response_text: str) -> str:
        """Extract SQL query from Gemini response"""
        response_text = response_text.strip()

        if response_text.startswith('```'):
            lines = response_text.split('\\n')
            lines = lines[1:]
            if lines and lines[-1].strip() == '```':
                lines = lines[:-1]
            response_text = '\\n'.join(lines)

        sql_query = response_text.strip()

        if not sql_query.upper().startswith('SELECT'):
            match = re.search(r'(SELECT\\s+.+)', sql_query, re.IGNORECASE | re.DOTALL)
            if match:
                sql_query = match.group(1)

        return sql_query

    def _validate_sql(self, sql_query: str) -> Tuple[bool, str]:
        """Validate SQL query"""
        if not sql_query.upper().strip().startswith('SELECT'):
            return False, "Only SELECT queries are allowed"

        dangerous_keywords = ['DROP', 'DELETE', 'UPDATE', 'INSERT', 'ALTER', 'CREATE', 'TRUNCATE']
        upper_query = sql_query.upper()
        for keyword in dangerous_keywords:
            if keyword in upper_query:
                return False, f"Query contains forbidden keyword: {keyword}"

        return True, "Valid"

    def execute_query(self, sql_query: str) -> Tuple:
        """Execute SQL query and return results"""
        try:
            conn = sqlite3.connect(self.db_path)
            df = pd.read_sql_query(sql_query, conn)
            conn.close()
            return df, "success"
        except Exception as e:
            return None, f"Error executing query: {str(e)}"

    def get_example_queries(self) -> List[str]:
        """Return list of example queries"""
        return [
            "Show all patients",
            "How many patients per site?",
            "List patients with severe adverse events",
            "Show patients older than 65",
            "Find lab results with outliers",
            "Count adverse events by severity"
        ]

print("✅ Gemini-powered NL-to-SQL module created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/nl_to_sql.py', 'w') as f:
    f.write(nl_to_sql_code)

# (Add data_quality_checker.py and query_generator.py - keeping your existing code)

print("✅ All modules created successfully!")

# ==========================================
# CELL 5: Create Enhanced Gradio App with Table Generator
# ==========================================

app_code = '''import gradio as gr
import sqlite3
import pandas as pd
import plotly.express as px
from datetime import datetime
import sys
import os

sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from nl_to_sql import SimpleNLtoSQL
from table_generator import DynamicTableGenerator

DB_PATH = '/content/drive/MyDrive/ClinicalTrialApp/clinical_trial.db'

nl_sql = None
table_gen = DynamicTableGenerator(DB_PATH)

def configure_api(api_key):
    global nl_sql
    if not api_key or api_key.strip() == "":
        return "⚠️ Please enter your Gemini API key", False

    try:
        nl_sql = SimpleNLtoSQL(DB_PATH, api_key=api_key.strip())
        return "✅ Gemini API configured! You can now use the query interface.", True
    except Exception as e:
        return f"❌ Error: {str(e)}", False

def execute_nl_query(user_query, confirmation):
    global nl_sql

    if not nl_sql:
        return "⚠️ Please configure Gemini API first!", None, "", gr.update(visible=False)

    if not user_query or user_query.strip() == "":
        return "Please enter a query.", None, "", gr.update(visible=False)

    try:
        sql_query, status, explanation = nl_sql.parse_query(user_query)

        if status == "success" and sql_query:
            if not confirmation:
                return (
                    f"🔍 Generated SQL Query:\\n\\n{sql_query}\\n\\n" +
                    f"💡 Explanation:\\n{explanation}\\n\\n" +
                    "⚠️ Please review and check the confirmation box to execute.",
                    None,
                    sql_query,
                    gr.update(visible=True)
                )

            df_result, exec_status = nl_sql.execute_query(sql_query)

            if exec_status == "success":
                result_text = f"✅ Query executed!\\n\\n📊 Found {len(df_result)} results"
                return result_text, df_result, sql_query, gr.update(visible=False)
            else:
                return f"❌ Error: {exec_status}", None, sql_query, gr.update(visible=False)
        else:
            return f"⚠️ {explanation}", None, sql_query or "", gr.update(visible=False)

    except Exception as e:
        return f"❌ Error: {str(e)}", None, "", gr.update(visible=False)

def create_custom_table(table_name, columns_text, num_rows):
    """Create a new custom table"""
    if not table_name or not columns_text:
        return "⚠️ Please provide table name and columns!", None

    # Parse columns (format: col1:TEXT, col2:INTEGER, col3:DATE)
    try:
        schema = {}
        for line in columns_text.strip().split('\\n'):
            if ':' in line:
                col, dtype = line.split(':')
                schema[col.strip()] = dtype.strip()

        if not schema:
            return "⚠️ Invalid column format! Use: column_name:DATA_TYPE (one per line)", None

        # Create table
        success, msg = table_gen.create_table_from_schema(table_name, schema)
        if not success:
            return msg, None

        # Insert data
        success, data_msg = table_gen.insert_data(table_name, schema, int(num_rows))

        # Show preview
        conn = sqlite3.connect(DB_PATH)
        df = pd.read_sql_query(f'SELECT * FROM "{table_name}" LIMIT 10', conn)
        conn.close()

        return f"{msg}\\n{data_msg}\\n\\n✅ Table created with {num_rows} rows!", df

    except Exception as e:
        return f"❌ Error: {str(e)}", None

def list_all_tables():
    """List all tables in database"""
    tables = table_gen.get_all_tables()
    return "\\n".join([f"• {t}" for t in tables])

with gr.Blocks(title="Clinical Trial Data Management", theme=gr.themes.Soft()) as app:

    gr.Markdown("""
    # 🏥 Clinical Trial Data Management System
    ### AI-Powered with Google Gemini + Dynamic Table Generator
    **Version 3.0**
    """)

    with gr.Tab("🔑 API Configuration"):
        gr.Markdown("### Configure Gemini API")

        api_key_input = gr.Textbox(
            label="🔑 Enter Gemini API Key",
            type="password",
            placeholder="Get from https://aistudio.google.com/apikey"
        )
        api_config_btn = gr.Button("⚙️ Configure API", variant="primary")
        api_status = gr.Textbox(label="API Status", lines=2)
        api_configured = gr.State(False)

        api_config_btn.click(
            fn=configure_api,
            inputs=[api_key_input],
            outputs=[api_status, api_configured]
        )

    with gr.Tab("🔍 Query Interface"):
        gr.Markdown("### Ask questions in plain English - Powered by Gemini 2.0")

        with gr.Row():
            with gr.Column(scale=2):
                query_input = gr.Textbox(
                    label="Your question:",
                    placeholder="e.g., Show patients with severe adverse events",
                    lines=2
                )

                confirmation_checkbox = gr.Checkbox(
                    label="✅ I reviewed the SQL and want to execute it",
                    value=False,
                    visible=False
                )

                with gr.Row():
                    query_btn = gr.Button("🚀 Generate SQL", variant="primary")
                    clear_btn = gr.Button("🗑️ Clear")

        query_output = gr.Textbox(label="Results", lines=8)
        sql_output = gr.Code(label="Generated SQL", language="sql")
        results_df = gr.Dataframe(label="Data", wrap=True)

        query_btn.click(
            fn=execute_nl_query,
            inputs=[query_input, confirmation_checkbox],
            outputs=[query_output, results_df, sql_output, confirmation_checkbox]
        )

        clear_btn.click(
            fn=lambda: ("", None, "", False, gr.update(visible=False)),
            outputs=[query_output, results_df, sql_output, confirmation_checkbox, confirmation_checkbox]
        )

    with gr.Tab("🆕 Create Custom Tables"):
        gr.Markdown("""
        ### 🎨 Dynamic Table Generator
        Create your own custom tables with automatic fake data generation!
        """)

        with gr.Row():
            with gr.Column():
                new_table_name = gr.Textbox(
                    label="📋 Table Name",
                    placeholder="e.g., custom_data"
                )

                columns_input = gr.Textbox(
                    label="📝 Columns (one per line)",
                    placeholder="column1:TEXT\\ncolumn2:INTEGER\\ncolumn3:DATE\\ncolumn4:REAL",
                    lines=8
                )

                gr.Markdown("""
                **Supported Data Types:**
                - TEXT (for names, emails, addresses, etc.)
                - INTEGER (for numbers, ages, IDs)
                - REAL (for decimals, prices, salaries)
                - DATE (for dates)
                - BOOLEAN (for yes/no values)

                **Smart Column Detection Examples:**
                - Columns with "age" → generates ages 18-85
                - Columns with "email" → generates valid emails
                - Columns with "phone/mobile" → generates phone numbers
                - Columns with "salary/price" → generates realistic amounts
                - Columns with "date/birth/dob" → generates birth dates
                - Columns with "hire/join" → generates employment dates
                - Columns with "department" → generates dept names
                - Columns with "status" → generates status values
                - Columns with "gender" → generates M/F values

                **💡 Pro Tips:**
                1. Use descriptive column names for better data
                2. Column name "user_age" will generate realistic ages
                3. Column name "emp_salary" will generate salary ranges
                4. Column name "birth_date" will generate birth dates
                """)

                gr.Markdown("""
                ---
                **📝 Quick Examples:**

                **Employee Table:**
                ```
                emp_name:TEXT
                emp_email:TEXT
                emp_age:INTEGER
                emp_salary:REAL
                department:TEXT
                hire_date:DATE
                status:TEXT
                ```

                **Product Table:**
                ```
                product_name:TEXT
                product_price:REAL
                category:TEXT
                stock_quantity:INTEGER
                ```

                **Customer Table:**
                ```
                customer_name:TEXT
                customer_email:TEXT
                phone_number:TEXT
                address:TEXT
                city:TEXT
                ```
                """)

                num_rows_input = gr.Slider(
                    minimum=1,
                    maximum=1000,
                    value=100,
                    step=1,
                    label="📊 Number of Rows to Generate"
                )

                create_table_btn = gr.Button("🎯 Create Table & Generate Data", variant="primary")

            with gr.Column():
                gr.Markdown("### 📚 Existing Tables")
                tables_list = gr.Textbox(
                    label="Current Tables in Database",
                    lines=15,
                    interactive=False
                )
                refresh_tables_btn = gr.Button("🔄 Refresh List")

        create_output = gr.Textbox(label="Status", lines=5)
        preview_df = gr.Dataframe(label="Preview (First 10 Rows)", wrap=True)

        create_table_btn.click(
            fn=create_custom_table,
            inputs=[new_table_name, columns_input, num_rows_input],
            outputs=[create_output, preview_df]
        )

        refresh_tables_btn.click(
            fn=list_all_tables,
            outputs=[tables_list]
        )

        # Auto-load tables on tab load
        app.load(fn=list_all_tables, outputs=[tables_list])

    with gr.Tab("📊 Dashboard"):
        gr.Markdown("### Analytics & Metrics")

        def create_dashboard():
            try:
                conn = sqlite3.connect(DB_PATH)

                # Get stats
                stats = {
                    'total_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients", conn).iloc[0]['count'],
                    'active_patients': pd.read_sql_query("SELECT COUNT(*) as count FROM patients WHERE status = 'Active'", conn).iloc[0]['count'],
                    'total_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events", conn).iloc[0]['count'],
                    'severe_aes': pd.read_sql_query("SELECT COUNT(*) as count FROM adverse_events WHERE severity = 'Severe'", conn).iloc[0]['count'],
                }

                summary = f"""
CLINICAL TRIAL DASHBOARD
Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}

KEY METRICS:

👥 Patients:
   • Total: {stats['total_patients']}
   • Active: {stats['active_patients']} ({stats['active_patients']/stats['total_patients']*100:.1f}%)

⚠️ Safety:
   • Total AEs: {stats['total_aes']}
   • Severe AEs: {stats['severe_aes']}
"""

                site_dist = pd.read_sql_query("SELECT site_id, COUNT(*) as count FROM patients GROUP BY site_id", conn)
                fig1 = px.bar(site_dist, x='site_id', y='count', title="Patients by Site")

                treatment_dist = pd.read_sql_query("SELECT treatment_arm, COUNT(*) as count FROM patients GROUP BY treatment_arm", conn)
                fig2 = px.pie(treatment_dist, values='count', names='treatment_arm', title="Treatment Arms")

                conn.close()

                return summary, fig1, fig2

            except Exception as e:
                return f"❌ Error: {str(e)}", None, None

        refresh_btn = gr.Button("🔄 Refresh", variant="primary")

        dashboard_summary = gr.Textbox(label="Metrics", lines=20)

        with gr.Row():
            chart1 = gr.Plot(label="Patients by Site")
            chart2 = gr.Plot(label="Treatment Arms")

        refresh_btn.click(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

        app.load(
            fn=create_dashboard,
            outputs=[dashboard_summary, chart1, chart2]
        )

    gr.Markdown("""
    ---
    **v3.0** | Powered by Google Gemini | Data in Google Drive | Dynamic Table Generator

    💾 Location: `/content/drive/MyDrive/ClinicalTrialApp/`

    🎯 **New Feature**: Create custom tables with automatic fake data generation!
    """)

print("✅ Enhanced Gradio app created!")
'''

with open('/content/drive/MyDrive/ClinicalTrialApp/app_gradio.py', 'w') as f:
    f.write(app_code)

print("✅ app_gradio.py saved successfully!")

# ==========================================
# CELL 6: Instructions
# ==========================================

print("""
╔═══════════════════════════════════════════════════════════╗
║    🎉 INTEGRATED APP WITH TABLE GENERATOR READY!          ║
╚═══════════════════════════════════════════════════════════╝

📝 HOW TO GET YOUR GEMINI API KEY:

1. 🌐 Go to: https://aistudio.google.com/apikey
2. 🔑 Click "Create API Key" button
3. 📋 Copy your API key
4. ✅ Paste it in the app when it launches
5. ⚙️ Click "Configure API" button

🎯 NEW FEATURES:
   ✅ AI SQL generation (Gemini 2.0)
   ✅ Query confirmation (safety first!)
   ✅ Interactive dashboard
   ✨ Dynamic table creation with fake data!
   ✨ Custom schema definition
   ✨ Automatic data type detection
   ✨ Real-time table listing

💾 DATA: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this notebook running!
""")

# ==========================================
# CELL 7: Launch the App
# ==========================================

import os
os.chdir('/content/drive/MyDrive/ClinicalTrialApp')

import sys
sys.path.append('/content/drive/MyDrive/ClinicalTrialApp')

from app_gradio import app

app.launch(
    share=True,
    debug=True,
    server_name="0.0.0.0",
    server_port=7860
)

print("""
╔═══════════════════════════════════════════════════════════╗
║  🎉 APP IS RUNNING!                                       ║
╚═══════════════════════════════════════════════════════════╝

🔑 SETUP:
   1. Click the public URL above
   2. Enter your Gemini API key in "API Configuration" tab
   3. Click "Configure API"
   4. Start using all features!

✨ TABS AVAILABLE:
   1️⃣ API Configuration - Setup Gemini
   2️⃣ Query Interface - Ask questions in plain English
   3️⃣ Create Custom Tables - NEW! Generate tables dynamically
   4️⃣ Dashboard - View analytics

🎨 EXAMPLE: CREATE CUSTOM TABLE
   Table Name: employee_data
   Columns:
   emp_name:TEXT
   emp_email:TEXT
   emp_age:INTEGER
   emp_salary:REAL
   department:TEXT
   hire_date:DATE
   status:TEXT

   Set rows to 100 → Click "Create Table & Generate Data"

   This will create realistic data:
   - emp_age: Ages between 18-85
   - emp_salary: Salaries $30,000-$200,000
   - hire_date: Employment dates within last 10 years
   - department: Sales, Engineering, HR, etc.
   - status: Active, Inactive, etc.

   Then query it: "show me employees older than 40"

💾 DATA SAVED IN: /content/drive/MyDrive/ClinicalTrialApp/

⚠️ Keep this cell running to use the app!

🔄 To restart: Run Cell 7 again
""")